In [1]:
from src.api import DataAPI , Predictor

DataAPI.update()
DataAPI.prepare_train_data()
Predictor.update_factors()

Update Files
Mon Aug  5 16:52:40 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.02 Secs
Mon Aug  5 16:52:40 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.04 Secs
Mon Aug  5 16:52:40 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.03 Secs
Mon Aug  5 16:52:44 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 3.17 Secs
Mon Aug  5 16:52:50 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 5.85 Secs
Mon Aug  5 16:52:51 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240731.feather Done! Cost 0.12 Secs
Mon Aug  5 16:52:52 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240801.feather Done! Cost 0.11 Secs
Mon Aug  5 16:53:06 2024 : Updated ~ data\DataBase\DB_benchmark\csi300\2024\csi300.20240729.feather Done! Cost 0.01 Secs
Mon Aug  5 16:53:06 2024 : Updated ~ data\DataBase\DB_benchmark\csi500\2024\csi500.20240729.feath

In [1]:
from src.api import Predictor
DataAPI.prepare_train_data()
Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [2]:
from src.api import ModelTestor
ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
